In [1]:
!pip install datasets
from datasets import get_dataset_config_names

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 11.8 MB/s eta 0:00:00


In [2]:
xtreme_subsets = get_dataset_config_names("xtreme")
print(f"Xtreme has {len(xtreme_subsets)} configurations")

Xtreme has 183 configurations


In [3]:
panx_subsets = [s for s in xtreme_subsets if s.startswith('PAN')]
panx_subsets[:3]
# suffix has ISO 639-1 language code

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg']

In [4]:
# language code for german - PAN-X.de
from datasets import load_dataset
load_dataset("xtreme", name='PAN-X.de')

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
})

In [5]:
from collections import defaultdict
from datasets import DatasetDict

In [6]:
langs = ["de", "fr", "it", "en"]
fracs = [0.629, 0.229, 0.084, 0.059]

In [7]:
# Return a DatasetDict if a key doesnt exist
panx_ch = defaultdict(DatasetDict)

In [8]:
panx_ch

defaultdict(datasets.dataset_dict.DatasetDict, {})

In [9]:
for lang, frac in zip(langs, fracs):
  # Load multilingual corpus
  ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
  # Shuffle and downsample each split according to spoken proportion
  for split in ds:
    panx_ch[lang][split] = (
        ds[split].shuffle(seed=0).select(range(int(frac*ds[split].num_rows)))
    )

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [10]:
panx_ch

defaultdict(datasets.dataset_dict.DatasetDict,
            {'de': DatasetDict({
                 train: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 12580
                 })
                 validation: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 6290
                 })
                 test: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 6290
                 })
             }),
             'fr': DatasetDict({
                 train: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 4580
                 })
                 validation: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 2290
                 })
                 test: Dataset({
                     features: ['tokens', 'ner_tags', 'la

In [11]:
import pandas as pd

In [12]:
pd.DataFrame({lang: [panx_ch[lang]["train"].num_rows] for lang in langs}, index=["Number of Training examples"])

,de,fr,it,en
Number of Training examples,12580,4580,1680,1180


In [13]:
element = panx_ch["de"]["train"][0]
element

{'tokens': ['2.000',
  'Einwohnern',
  'an',
  'der',
  'Danziger',
  'Bucht',
  'in',
  'der',
  'polnischen',
  'Woiwodschaft',
  'Pommern',
  '.'],
 'ner_tags': [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0],
 'langs': ['de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de',
  'de']}

In [14]:
for key, val in element.items():
  print(f"{key}: {val}")

tokens: ['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in', 'der', 'polnischen', 'Woiwodschaft', 'Pommern', '.']
ner_tags: [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]
langs: ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']


In [15]:
panx_ch["de"]["train"].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None),
 'langs': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [16]:
# decoding the ner_tags
for key, val in panx_ch["de"]["train"].features.items():
  print(f"{key}: {val}")

tokens: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
ner_tags: Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)
langs: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)


In [17]:
tags = panx_ch["de"]["train"].features["ner_tags"].feature
tags

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)

In [18]:
def create_tag_names(batch):
  return {"ner_tags_str": [tags.int2str(idx) for idx in batch["ner_tags"]]}

In [19]:
panx_de = panx_ch["de"].map(create_tag_names)

Map:   0%|          | 0/12580 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

In [20]:
panx_de.items()

dict_items([('train', Dataset({
    features: ['tokens', 'ner_tags', 'langs', 'ner_tags_str'],
    num_rows: 12580
})), ('validation', Dataset({
    features: ['tokens', 'ner_tags', 'langs', 'ner_tags_str'],
    num_rows: 6290
})), ('test', Dataset({
    features: ['tokens', 'ner_tags', 'langs', 'ner_tags_str'],
    num_rows: 6290
}))])

In [21]:
panx_de['train'].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None),
 'langs': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags_str': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [22]:
de_example = panx_de["train"][0]
pd.DataFrame([de_example["tokens"], de_example["ner_tags_str"]], index=["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8,9,10,11
Tokens,2.000,Einwohnern,an,der,Danziger,Bucht,in,der,polnischen,Woiwodschaft,Pommern,.
Tags,O,O,O,O,B-LOC,I-LOC,O,O,B-LOC,B-LOC,I-LOC,O


In [23]:
from collections import Counter

In [24]:
split2freqs = defaultdict(Counter)
for split, dataset in panx_de.items():
  for row in dataset["ner_tags_str"]:
    for tag in row:
      if tag.startswith("B"):
        tag_type = tag.split("-")[1]
        split2freqs[split][tag_type]+=1

pd.DataFrame.from_dict(split2freqs, orient="index")

,LOC,ORG,PER
train,6186,5366,5810
validation,3172,2683,2893
test,3180,2573,3071


In [25]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.5 MB/s eta 0:00:00


In [26]:
from transformers import AutoTokenizer

In [27]:
bert_model_name = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"

bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

In [28]:
text = "Tonit Kumar loves New York!"
bert_tokens = bert_tokenizer(text)
xlmr_tokens = xlmr_tokenizer(text)

In [29]:
bert_tokens, xlmr_tokens

({'input_ids': [101, 17752, 1204, 9392, 7871, 1203, 1365, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [0, 98765, 18, 18385, 5161, 7, 2356, 5753, 38, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]})

In [30]:
bert_tokens.tokens(), xlmr_tokens.tokens()

(['[CLS]', 'Toni', '##t', 'Kumar', 'loves', 'New', 'York', '!', '[SEP]'],
 ['<s>', '▁Toni', 't', '▁Kumar', '▁love', 's', '▁New', '▁York', '!', '</s>'])

In [31]:
pd.DataFrame([bert_tokens.tokens(), bert_tokens["input_ids"], xlmr_tokens.tokens(), xlmr_tokens["input_ids"]],
             index=["bert tokens", "bert ids", "xlmr tokens", "xlmr ids"])

,0,1,2,3,4,5,6,7,8,9
bert tokens,[CLS],Toni,##t,Kumar,loves,New,York,!,[SEP],None
bert ids,101,17752,1204,9392,7871,1203,1365,106,102,None
xlmr tokens,<s>,▁Toni,t,▁Kumar,▁love,s,▁New,▁York,!,</s>
xlmr ids,0,98765,18,18385,5161,7,2356,5753,38,2


In [32]:
"".join(xlmr_tokens.tokens()).replace("\u2581", " ")

'<s> Tonit Kumar loves New York!</s>'

# Creating a custom model for Token Classification

In [33]:
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel

In [34]:
print(XLMRobertaConfig)

<class 'transformers.models.xlm_roberta.configuration_xlm_roberta.XLMRobertaConfig'>


In [35]:
class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
  config_class = XLMRobertaConfig
  def __init__(self, config):
    super().__init__(config)
    self.num_labels = config.num_labels

    # load model body
    self.roberta = RobertaModel(config, add_pooling_layer=False)

    # set up token classification head
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    # load and initialize weights
    self.init_weights()

  def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):

    # use model body to get encoder representation
    outputs = self.roberta(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, **kwargs)

    # apply classifier to encoder representation
    sequence_output = self.dropout(outputs[0])
    logits = self.classifier(sequence_output)

    # calculate loss
    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

    # return model output object
    return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions)

In [36]:
tags, tags.num_classes

(ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None),
 7)

In [37]:
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}

In [38]:
from transformers import AutoConfig

In [39]:
xlmr_config = AutoConfig.from_pretrained(xlmr_model_name,
                                         num_labels=tags.num_classes,
                                         id2label=index2tag, label2id=tag2index)

In [40]:
import torch

In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [42]:
xlmr_model = (XLMRobertaForTokenClassification.from_pretrained(xlmr_model_name, config=xlmr_config).to(device))

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
input_ids = xlmr_tokenizer.encode(text, return_tensors="pt")
input_ids

tensor([[    0, 98765,    18, 18385,  5161,     7,  2356,  5753,    38,     2]])

In [44]:
xlmr_tokens.tokens()

['<s>', '▁Toni', 't', '▁Kumar', '▁love', 's', '▁New', '▁York', '!', '</s>']

In [45]:
pd.DataFrame([xlmr_tokens.tokens(), input_ids[0].numpy()], index=["tokens", "input_ids"])

,0,1,2,3,4,5,6,7,8,9
tokens,<s>,▁Toni,t,▁Kumar,▁love,s,▁New,▁York,!,</s>
input_ids,0,98765,18,18385,5161,7,2356,5753,38,2


In [46]:
outputs = xlmr_model(input_ids.to(device))
outputs, outputs[0]

(TokenClassifierOutput(loss=None, logits=tensor([[[-0.2924, -0.2692,  0.0780, -0.5065,  0.3344, -0.2148,  0.0259],
          [-0.1081, -0.0883,  0.1520, -0.5038,  0.1586, -0.0024, -0.1605],
          [-0.1255, -0.1029,  0.1002, -0.4987,  0.1998, -0.0407,  0.0115],
          [-0.0013, -0.1187,  0.1565, -0.4175,  0.2376,  0.0435, -0.0800],
          [-0.0864, -0.0468,  0.0075, -0.3345,  0.2751,  0.1515, -0.0841],
          [-0.0889, -0.1155,  0.0097, -0.4282,  0.1984, -0.0057, -0.0306],
          [-0.1103, -0.0163,  0.1111, -0.3558,  0.2268,  0.0103, -0.1534],
          [-0.0524, -0.0414,  0.1502, -0.3069,  0.2124,  0.1145, -0.0729],
          [-0.1770, -0.1041,  0.1710, -0.3749,  0.2060,  0.0062, -0.0908],
          [-0.2993, -0.2686,  0.1062, -0.5242,  0.3780, -0.2220,  0.0742]]],
        device='cuda:0', grad_fn=<ViewBackward0>), hidden_states=None, attentions=None),
 tensor([[[-0.2924, -0.2692,  0.0780, -0.5065,  0.3344, -0.2148,  0.0259],
          [-0.1081, -0.0883,  0.1520, -0.503

In [47]:
outputs = outputs.logits
predictions = torch.argmax(outputs, dim=-1)
print(predictions)
print(f"Number of tokens in sequence: {len(xlmr_tokens.tokens())}")
print(f"Shape of outputs: {outputs.size()}")

tensor([[4, 4, 4, 4, 4, 4, 4, 4, 4, 4]], device='cuda:0')
Number of tokens in sequence: 10
Shape of outputs: torch.Size([1, 10, 7])


In [48]:
preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
pd.DataFrame([xlmr_tokens.tokens(), preds], index=["tokens", "predictions"])

,0,1,2,3,4,5,6,7,8,9
tokens,<s>,▁Toni,t,▁Kumar,▁love,s,▁New,▁York,!,</s>
predictions,I-ORG,I-ORG,I-ORG,I-ORG,I-ORG,I-ORG,I-ORG,I-ORG,I-ORG,I-ORG


In [49]:
def tag_text(text, tags, model, tokenizer):
  # Get tokens with special characters
  tokens = tokenizer(text).tokens()

  # encode the sequence into ids
  input_ids = xlmr_tokenizer(text, return_tensors="pt").input_ids.to(device)

  # get predictions over 7 classes
  outputs = model(input_ids)[0]

  # take argmax to get most likely class per token
  predictions = torch.argmax(ot=outputs, dim=-1)

  # convert to dataframe
  preds = [tags.names[p] for p in predictions[0].cput().numpy()]

  return pd.DataFrame([tokens, preds], index=["Tokens", "Tags"])

# Tokenizing text for NER

In [51]:
words, labels = de_example["tokens"], de_example["ner_tags"]
words, labels

(['2.000',
  'Einwohnern',
  'an',
  'der',
  'Danziger',
  'Bucht',
  'in',
  'der',
  'polnischen',
  'Woiwodschaft',
  'Pommern',
  '.'],
 [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0])

In [53]:
tokenized_input = xlmr_tokenizer(de_example["tokens"], is_split_into_words=True)
tokens = xlmr_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
pd.DataFrame([tokens], index=["Tokens"])

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>
